In [1]:
from pyvi import ViTokenizer
import numpy as np
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
import warnings
warnings.filterwarnings('ignore')

# Tập data chatbot

In [2]:
file_tranning = ['./dữ liệu chatbot - dự án cuối kì/bạn bè.txt',
'./dữ liệu chatbot - dự án cuối kì/các câu hỏi phức tạp.txt',
'./dữ liệu chatbot - dự án cuối kì/đất nước.txt',
'./dữ liệu chatbot - dự án cuối kì/địa chỉ.txt',
'./dữ liệu chatbot - dự án cuối kì/du lịch.txt',
'./dữ liệu chatbot - dự án cuối kì/gia đình.txt',
'./dữ liệu chatbot - dự án cuối kì/giải trí.txt',
'./dữ liệu chatbot - dự án cuối kì/học tập.txt',
'./dữ liệu chatbot - dự án cuối kì/nghề nghiệp.txt',
'./dữ liệu chatbot - dự án cuối kì/nghỉ lễ.txt',
'./dữ liệu chatbot - dự án cuối kì/người yêu.txt',
'./dữ liệu chatbot - dự án cuối kì/robot.txt',
'./dữ liệu chatbot - dự án cuối kì/shoping.txt',
'./dữ liệu chatbot - dự án cuối kì/tán gẫu.txt',
'./dữ liệu chatbot - dự án cuối kì/tdtu.txt',
'./dữ liệu chatbot - dự án cuối kì/thông tin cá nhân.txt',
'./dữ liệu chatbot - dự án cuối kì/trò chuyện về đi ăn.txt']

In [3]:
question_train = []
answer_train = []

for i in range(len(file_tranning)):
    with open(file_tranning[i], encoding='UTF-8') as f:
        train_lines = f.readlines()
        for line in train_lines:
            tmp = line.split("__eou__")
            question_train.append(tmp[0].strip()) # strip(): Loại bỏ whitespace đầu cuối string
            answer_train.append(tmp[1].strip())

# Preprocessing

In [4]:
# Xóa question và answer khi answer empty string
list_index_empty_answer = []

In [5]:
for i in range(len(answer_train)):
    if(answer_train[i] == ""):
        list_index_empty_answer.append(i)

In [6]:
print(list_index_empty_answer)

[423, 424, 425, 426, 427, 428, 486, 487, 488, 489, 490, 632, 633, 648, 649, 650, 651, 652, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1229, 1230, 1231, 1232, 1233, 1814, 1815, 1816, 1817, 1871, 1872, 1873, 1874, 2372, 2373, 2374, 2375, 2376, 2377, 2653, 2654, 2703, 2704, 2705, 2706, 3240, 3241, 3242, 3243, 3244, 3289, 3290, 3291, 3292, 3293, 4195, 4196, 4197, 4198, 4199, 5030, 5031, 5032, 5033, 5034, 5035, 5405]


In [7]:
for i in range(len(list_index_empty_answer)):
    del answer_train[list_index_empty_answer[i] - i]
    del question_train[list_index_empty_answer[i] - i]

In [8]:
import string

In [9]:
def text_process(mess):
    # chuyển về chữ thường
    mess = mess.lower()
    
    # xóa dấu câu
    mess = [char for char in mess if char not in string.punctuation]
    mess = ''.join(mess)
    
    # replace whitespace
    mess = mess.replace("   ", " ")
    mess = mess.replace("  ", " ")
    
    # Word Segmentation
    mess = ViTokenizer.tokenize(mess)
    
    return mess

In [10]:
for i in range(len(question_train)):
    question_train[i] = text_process(question_train[i])
    answer_train[i] = text_process(answer_train[i])

In [11]:
print(question_train[0])
print(answer_train[0])

thích đánh_lộn không
ngon nhà_vô


In [12]:
data = question_train
data_answer = answer_train

# MAKE VOCABULARY

In [13]:
# Đếm số lần xuất hiện của 1 từ
word2count = {}

for line in data:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
            
for line in data_answer:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [14]:
len(word2count)

4857

In [15]:
word2index = {}
word_number = 0
for word, count in word2count.items():
    word2index[word] = word_number
    word_number += 1

In [16]:
len(word2index)

4857

In [18]:
# word2index

In [19]:
data_answer = ["<SOS> " + answer + " <EOS>" for answer in data_answer]

In [20]:
data_answer[0:2]

['<SOS> ngon nhà_vô <EOS>', '<SOS> chấp lun 2 mạng đầu <EOS>']

In [21]:
tokens = ['<EOS>', '<OUT>', '<SOS>']

VOCAB_SIZE = len(word2index)

for token in tokens:
    word2index[token] = VOCAB_SIZE
    VOCAB_SIZE += 1

In [22]:
index2word = {w:v for v, w in word2index.items()}
len(index2word)

4860

In [23]:
# index2word

# ONE-HOT VECTORIZER, PADDING

In [24]:
# chuyển các câu trong question thành vector
encoder_input_data = []
for line in data:
    tmp = []
    for word in line.split():
        if word not in word2index:
            tmp.append(word2index['<OUT>'])
        else:
            tmp.append(word2index[word])
        
    encoder_input_data.append(tmp)

In [26]:
# encoder_input_data

In [27]:
# chuyển các câu trong answer thành vector
decoder_input_data = []
for line in data_answer:
    tmp = []
    for word in line.split():
        if word not in word2index:
            tmp.append(word2index['<OUT>'])
        else:
            tmp.append(word2index[word])        
    decoder_input_data.append(tmp)

In [29]:
# decoder_input_data

In [30]:
MAX_LEN = 20
# kỹ thuật để cho độ dài vector đều như nhau
encoder_input_data = pad_sequences(encoder_input_data, MAX_LEN, padding='post', truncating='post')
decoder_input_data = pad_sequences(decoder_input_data, MAX_LEN, padding='post', truncating='post')

In [32]:
encoder_input_data

array([[   0,    1,    2, ...,    0,    0,    0],
       [   3,    4,    2, ...,    0,    0,    0],
       [   5,    6,    7, ...,    0,    0,    0],
       ...,
       [  21,   71,  129, ...,    0,    0,    0],
       [  21,   71,  129, ...,    0,    0,    0],
       [  11, 1226,    6, ...,    0,    0,    0]])

In [42]:
# output
decoder_target_data = []
for i in decoder_input_data:
    decoder_target_data.append(i[1:])

In [43]:
decoder_target_data

[array([ 894, 2558, 4857,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]),
 array([2559, 2514,  454,  492,  475, 4857,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]),
 array([   5,  813,    8,   15, 4857,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]),
 array([   5,    8,    9,  593, 4857,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]),
 array([  12, 2560, 2561, 4857,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]),
 array([2562,   15, 4857,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]),
 array([ 337,  699,   15, 4857,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]),
 array([ 129,   22,   15, 4857,    0,    0,    0,    0,    0,    0,    0,
  

In [44]:
decoder_target_data = pad_sequences(decoder_target_data, MAX_LEN, padding='post', truncating='post')

In [45]:
decoder_target_data

array([[ 894, 2558, 4857, ...,    0,    0,    0],
       [2559, 2514,  454, ...,    0,    0,    0],
       [   5,  813,    8, ...,    0,    0,    0],
       ...,
       [ 105,  225,  164, ...,    0,    0,    0],
       [   2, 4857,    0, ...,    0,    0,    0],
       [ 435, 4857,    0, ...,    0,    0,    0]])

In [37]:
# (sample, timestep, feature)
decoder_target_data = to_categorical(decoder_target_data, len(word2index))

In [38]:
print(decoder_target_data.shape)

(5537, 20, 4860)


# MODEL

In [39]:
VOCAB_SIZE

4860

In [40]:
# đại diện cho chuỗi các từ, được encode bằng index của chúng trong index2word
# nên nhúng các index qua Embedding layer
embed = Embedding(VOCAB_SIZE + 1, output_dim=50, input_length=MAX_LEN, trainable=True)

In [41]:
latent_dim = 128

# encoder: xử lý chuỗi đầu vào và trả về state bên trong của nó
encoder_inputs = Input(shape=(MAX_LEN, ))
encoder_embed = embed(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embed)

# Chỉ giữa lại các state: đóng vai trò là "context" hay "conditioning" của decoder trong bước tiếp theo.
encoder_states = [state_h, state_c]

# decoder: được train để predict các từ tiếp theo cho các từ trước đó của decoder_target_data
# using `encoder_states` as initial state.
decoder_inputs = Input(shape=(MAX_LEN, ))
decoder_embed = embed(decoder_inputs)

# thiết lập decoder trả về các chuỗi đầu ra đầy đủ, và trả về state bên trong. 
# Không sử dụng state trong train model
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embed, initial_state=encoder_states)

dense = Dense(VOCAB_SIZE, activation='softmax')

output = dense(decoder_outputs)


# Define the model
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], output)

model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 50)       243050      input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 20, 128), (N 91648       embedding[0][0]              

In [42]:
# Run training
EPOCHS = 100
BATCH_SIZE = 32

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          validation_split=0.2)

Epoch 1/100
139/139 [==============================] - 23s 143ms/step - loss: 5.3071 - acc: 0.5612 - val_loss: 2.4902 - val_acc: 0.6496
Epoch 2/100
139/139 [==============================] - 13s 92ms/step - loss: 2.7897 - acc: 0.6004 - val_loss: 2.1540 - val_acc: 0.7154
Epoch 3/100
139/139 [==============================] - 12s 84ms/step - loss: 2.4402 - acc: 0.6473 - val_loss: 2.1261 - val_acc: 0.7162
Epoch 4/100
139/139 [==============================] - 11s 83ms/step - loss: 2.3983 - acc: 0.6487 - val_loss: 2.1123 - val_acc: 0.7159
Epoch 5/100
139/139 [==============================] - 11s 82ms/step - loss: 2.4025 - acc: 0.6428 - val_loss: 2.1106 - val_acc: 0.7160
Epoch 6/100
139/139 [==============================] - 11s 83ms/step - loss: 2.3183 - acc: 0.6521 - val_loss: 2.1091 - val_acc: 0.7175
Epoch 7/100
139/139 [==============================] - 13s 94ms/step - loss: 2.3216 - acc: 0.6507 - val_loss: 2.1101 - val_acc: 0.7175
Epoch 8/100
139/139 [==============================] -

In [46]:
# Save model
model.save('NLP_LSTM_model_Encoder_Decoder.h5')

In [59]:
# Save vocab
import pickle

with open('word2index.pkl', 'wb') as f:
    pickle.dump(word2index, f)

with open('index2word.pkl', 'wb') as f:
    pickle.dump(index2word, f)